# 01B Data Cleansing Traps

## Imports

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

## Read Data

In [80]:
df = pd.read_csv('../west_nile/west_nile/input/train.csv')

## Read in codes_df and temperature_df

In [81]:
!ls ../Cleansed_Data/

X.pkl              codes_df.pkl       temperature_df.pkl y.pkl


In [82]:
with open('../Cleansed_Data/codes_df.pkl', 'rb') as f:
    codes_df = pickle.load(f)

with open('../Cleansed_Data/temperature_df.pkl', 'rb') as f:
    temperature_df = pickle.load(f)    

## Calculate Total Mosquitos Per Trap
- Fix this so we don't drop columns

In [83]:
# cols = [col for col in df.columns if col not in ['NumMosquitos', 'WnvPresent']]
# df = df.groupby(cols, as_index=False).sum()
# df['WnvPresent'] = (df.WnvPresent>0).astype(int)

## Join codes_df and temperature_df

In [84]:
df = df.join(temperature_df, on='Date')
df = df.join(codes_df, on='Date', rsuffix="_code_")

## Create boolean - WnvSpecies
- Discussion - Should we make it like this, or split out by species?
- I think we should set up the data for neural net?

In [85]:
wnvspeciesmap = {
    'CULEX PIPIENS/RESTUANS': 1,
    'CULEX RESTUANS': 1,
    'CULEX PIPIENS': 1,
    'CULEX TERRITANS': 0,
    'CULEX SALINARIUS': 0,
    'CULEX TARSALIS': 0,
    'CULEX ERRATICUS': 0,
    'UNSPECIFIED CULEX':1
}

df['WnvSpecies'] = df.Species.map(wnvspeciesmap)

## Extract Month from Date

In [86]:
df['Month'] = df.Date.str[5:7]

## One-hot encoding

In [87]:
df = pd.get_dummies(df, columns=['Month', 'Species'])

## Drop Unnecessary Columns

In [88]:
cols_drop = ['Address','Block','Street','Trap','AddressNumberAndStreet','AddressAccuracy','Date']
df = df.drop(labels=cols_drop, axis=1)

## Split X and y Variables

In [89]:
target = 'WnvPresent'
features = [col for col in df.columns if col != target]
y = df[target]
X = df[features]

## Export X and y Variables

In [90]:
with open('../Cleansed_Data/X.pkl','wb') as f:
    pickle.dump(X, f)
    
with open('../Cleansed_Data/y.pkl','wb') as f:
    pickle.dump(y, f)